# Vbench 框架测试笔记本

这个笔记本提供了一套全面的测试功能，用于验证Vbench框架的各个子模块是否能够正常工作。通过这个笔记本，您可以：

1. 测试数据集加载和处理功能
2. 测试模型构建和前向传播
3. 测试任务定义和执行
4. 测试训练器功能
5. 验证完整的训练和评估流程
6. 可视化模型性能和数据分布

让我们开始进行测试！

In [ ]:
# 导入必要的库
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import yaml
from pprint import pprint

# 添加项目根目录到路径
# 获取当前笔记本路径
current_dir = os.getcwd()
# 获取项目根目录
project_root = os.path.dirname(current_dir) if os.path.basename(current_dir) == 'test' else current_dir
if project_root not in sys.path:
    sys.path.append(project_root)

# 导入项目模块
try:
    from src.utils.config_utils import load_config, makedir, path_name, transfer_namespace
    from src.data_factory import build_dataset
    from src.model_factory import build_model
    from src.task_factory import build_task
    from src.trainer_factory import build_trainer
    from src.utils.metrics_utils import compute_metrics
    from src.utils.loss_utils import get_loss_function
    
    print("✅ 成功导入项目模块！")
except ImportError as e:
    print(f"❌ 导入模块失败: {str(e)}")
    print("请检查项目结构和安装依赖。")

## 测试环境设置

首先，我们将设置测试环境，包括必要的目录结构和配置文件

In [ ]:
# 创建必要的目录
test_dirs = [
    os.path.join(project_root, "results"),
    os.path.join(project_root, "data/processed"),
    os.path.join(project_root, "data/raw"),
    os.path.join(project_root, "save"),
    os.path.join(project_root, "test/results") 
]

for d in test_dirs:
    os.makedirs(d, exist_ok=True)
    print(f"📁 目录已准备: {d}")

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# 设置默认测试配置路径
default_config_path = os.path.join(project_root, "configs/demo/dummy_test.yaml")

# 检查配置文件是否存在
if os.path.exists(default_config_path):
    print(f"✅ 测试配置文件已存在: {default_config_path}")
else:
    print(f"❌ 未找到测试配置文件: {default_config_path}")
    print("将创建默认测试配置...")
    
    # 创建默认配置目录
    os.makedirs(os.path.dirname(default_config_path), exist_ok=True)
    
    # 创建一个默认的测试配置
    default_config = {
        "dataset": {
            "name": "DummyDataset",
            "args": {
                "task_type": "classification",
                "num_samples": 1000,
                "feature_dim": 10,
                "num_classes": 2
            }
        },
        "model": {
            "name": "DummyCNNModel",
            "args": {
                "input_channels": 1,
                "hidden_channels": [16, 32, 64],
                "output_dim": 2
            }
        },
        "task": {
            "name": "DummyClassificationTask",
            "args": {
                "num_classes": 2
            }
        },
        "trainer": {
            "name": "ModularTrainer",
            "args": {
                "epochs": 2,  # 测试用较少的轮次
                "batch_size": 32,
                "loss_fn": "cross_entropy",
                "metrics": ["accuracy", "precision", "recall", "f1"]
            }
        }
    }
    
    # 将配置写入文件
    with open(default_config_path, 'w') as f:
        yaml.dump(default_config, f, default_flow_style=False)
    
    print(f"✅ 已创建默认测试配置: {default_config_path}")

## 1. 数据工厂测试

测试数据集的加载和处理

In [ ]:
def test_data_factory():
    print("🔍 测试数据集工厂...")
    
    # 加载配置
    configs = load_config(default_config_path)
    
    # 构建数据集
    try:
        dataset = build_dataset(configs['dataset'])
        print(f"✅ 成功构建数据集: {dataset.__class__.__name__}")
        
        # 获取数据加载器
        train_loader = dataset.get_train_loader()
        val_loader = dataset.get_val_loader()
        test_loader = dataset.get_test_loader()
        
        # 打印数据加载器信息
        print(f"📊 训练集大小: {len(train_loader.dataset)} 样本")
        print(f"📊 验证集大小: {len(val_loader.dataset)} 样本")
        print(f"📊 测试集大小: {len(test_loader.dataset)} 样本")
        
        # 检查一个批次的数据
        x_batch, y_batch = next(iter(train_loader))
        print(f"📊 批次特征形状: {x_batch.shape}")
        print(f"📊 批次标签形状: {y_batch.shape}")
        
        # 可视化一批数据
        plt.figure(figsize=(10, 5))
        
        # 左图：特征数据可视化
        plt.subplot(1, 2, 1)
        plt.imshow(x_batch[0].reshape(x_batch.shape[1], -1).detach().numpy(), aspect='auto', cmap='viridis')
        plt.colorbar()
        plt.title('特征矩阵示例')
        plt.xlabel('特征')
        plt.ylabel('样本')
        
        # 右图：标签分布
        plt.subplot(1, 2, 2)
        class_counts = np.bincount(y_batch.numpy())
        plt.bar(range(len(class_counts)), class_counts)
        plt.title('标签分布')
        plt.xlabel('类别')
        plt.ylabel('样本数')
        plt.xticks(range(len(class_counts)))
        
        plt.tight_layout()
        plt.show()
        
        return {
            "success": True,
            "dataset": dataset,
            "train_loader": train_loader,
            "val_loader": val_loader,
            "test_loader": test_loader
        }
    
    except Exception as e:
        import traceback
        print(f"❌ 数据集测试失败: {str(e)}")
        traceback.print_exc()
        return {"success": False, "error": str(e)}

# 运行数据集测试
data_test_results = test_data_factory()

## 2. 模型工厂测试

测试模型的构建和前向传播

In [ ]:
def test_model_factory():
    print("🔍 测试模型工厂...")
    
    # 加载配置
    configs = load_config(default_config_path)
    
    try:
        # 构建模型
        model = build_model(configs['model'])
        print(f"✅ 成功构建模型: {model.__class__.__name__}")
        
        # 打印模型摘要
        if hasattr(model, 'summary'):
            print("📊 模型摘要:")
            print(model.summary())
        print(f"📊 模型参数数量: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
        
        # 如果我们有数据加载器，尝试模型前向传播
        if 'data_test_results' in globals() and data_test_results["success"]:
            train_loader = data_test_results["train_loader"]
            x_batch, _ = next(iter(train_loader))
            
            # 前向传播
            with torch.no_grad():
                outputs = model(x_batch)
                
            print(f"✅ 前向传播成功，输出形状: {outputs.shape}")
            
            # 可视化输出分布
            plt.figure(figsize=(10, 5))
            
            if outputs.shape[1] <= 10:  # 如果输出维度不太大
                # 输出激活值分布
                plt.subplot(1, 2, 1)
                output_np = outputs.detach().numpy()
                for i in range(outputs.shape[1]):
                    plt.hist(output_np[:, i], bins=20, alpha=0.5, label=f'Class {i}')
                plt.title('模型输出分布')
                plt.xlabel('激活值')
                plt.ylabel('频率')
                if outputs.shape[1] <= 5:  # 只有少量类别时显示图例
                    plt.legend()
                
                # 各类别平均激活值
                plt.subplot(1, 2, 2)
                plt.bar(range(outputs.shape[1]), outputs.mean(dim=0).detach().numpy())
                plt.title('各类别平均激活值')
                plt.xlabel('类别')
                plt.ylabel('平均激活值')
                plt.xticks(range(outputs.shape[1]))
                
                plt.tight_layout()
                plt.show()
            else:
                # 如果输出维度很大，只显示前10个维度
                plt.bar(range(10), outputs.mean(dim=0)[:10].detach().numpy())
                plt.title('前10个输出维度的平均激活值')
                plt.xlabel('输出维度')
                plt.ylabel('平均激活值')
                plt.show()
            
        return {"success": True, "model": model}
        
    except Exception as e:
        import traceback
        print(f"❌ 模型测试失败: {str(e)}")
        traceback.print_exc()
        return {"success": False, "error": str(e)}

# 运行模型测试
model_test_results = test_model_factory()

## 3. 任务工厂测试

测试任务的定义和执行

In [ ]:
def test_task_factory():
    print("🔍 测试任务工厂...")
    
    # 加载配置
    configs = load_config(default_config_path)
    
    try:
        # 需要先有模型才能测试任务
        if 'model_test_results' not in globals() or not model_test_results["success"]:
            print("⚠️ 模型尚未成功构建，先创建模型...")
            model = build_model(configs['model'])
        else:
            model = model_test_results["model"]
        
        # 将模型添加到任务配置
        task_config = configs['task'].copy()
        task_config['args'] = task_config.get('args', {}).copy()
        task_config['args']['model'] = model
        
        # 构建任务
        task = build_task(task_config)
        print(f"✅ 成功构建任务: {task.__class__.__name__}")
        
        # 测试任务功能
        if hasattr(task, 'get_loss_fn'):
            loss_fn = task.get_loss_fn()
            print(f"✅ 损失函数类型: {type(loss_fn).__name__}")
        
        # 如果有数据，尝试执行一个训练步骤
        if 'data_test_results' in globals() and data_test_results["success"]:
            train_loader = data_test_results["train_loader"]
            x_batch, y_batch = next(iter(train_loader))
            
            # 模型推理
            outputs = model(x_batch)
            
            # 如果任务有evaluate方法，测试评估
            if hasattr(task, 'evaluate'):
                print("📊 测试任务评估函数...")
                eval_results = task.evaluate(data_test_results["test_loader"])
                print("✅ 评估结果:")
                pprint(eval_results)
            
            # 如果任务有calculate_metrics方法，测试指标计算
            if hasattr(task, 'calculate_metrics'):
                print("📊 测试任务指标计算...")
                metrics = task.calculate_metrics(outputs, y_batch)
                print("✅ 计算指标:")
                pprint(metrics)
                
        return {"success": True, "task": task}
    
    except Exception as e:
        import traceback
        print(f"❌ 任务测试失败: {str(e)}")
        traceback.print_exc()
        return {"success": False, "error": str(e)}

# 运行任务测试
task_test_results = test_task_factory()

## 4. 训练器工厂测试

测试训练器的构建和简单训练

In [ ]:
def test_trainer_factory():
    print("🔍 测试训练器工厂...")
    
    # 加载配置
    configs = load_config(default_config_path)
    
    try:
        # 确保我们有任务和数据加载器
        if ('task_test_results' not in globals() or not task_test_results["success"] or
            'data_test_results' not in globals() or not data_test_results["success"]):
            print("⚠️ 任务或数据加载器尚未成功构建，无法测试训练器")
            return {"success": False, "error": "缺少任务或数据加载器"}
        
        task = task_test_results["task"]
        model = model_test_results["model"]
        dataset = data_test_results["dataset"]
        
        # 构建训练器
        trainer = build_trainer(configs['trainer'])
        print(f"✅ 成功构建训练器: {trainer.__class__.__name__}")
        
        # 为测试准备一个小型训练
        mini_epochs = 1  # 只训练一个周期用于测试
        configs['trainer']['args']['epochs'] = mini_epochs
        
        # 设置测试保存路径
        save_path = os.path.join(project_root, "test/results/trainer_test")
        os.makedirs(save_path, exist_ok=True)
        
        # 准备命名空间参数
        args_t = transfer_namespace(configs['trainer'].get('args', {}))
        args_m = transfer_namespace(configs['model'].get('args', {}))
        args_d = transfer_namespace(configs['dataset'].get('args', {}))
        args_task = transfer_namespace(configs['task'].get('args', {}))
        
        print(f"🚀 开始测试训练 (仅运行 {mini_epochs} 个周期)...")
        train_loader = data_test_results["train_loader"]
        val_loader = data_test_results["val_loader"]
        test_loader = data_test_results["test_loader"]
        
        # 执行训练
        result = trainer(
            dataset=dataset,
            model=model,
            task=task,
            train_loader=train_loader,
            val_loader=val_loader,
            test_loader=test_loader,
            configs=configs,
            args_t=args_t,
            args_m=args_m,
            args_d=args_d,
            args_task=args_task,
            save_path=save_path,
            iteration=0
        )
        
        print("\n✅ 训练测试完成")
        print("📊 训练结果:")
        pprint(result)
        
        # 如果有测试结果，绘制指标图
        if isinstance(result, dict) and len(result) > 0:
            # 筛选数值型指标
            metrics = {k: v for k, v in result.items() if isinstance(v, (int, float))}
            if metrics:
                plt.figure(figsize=(10, 6))
                plt.bar(metrics.keys(), metrics.values())
                plt.title('测试评估指标')
                plt.ylabel('值')
                plt.xticks(rotation=45)
                plt.grid(axis='y', linestyle='--', alpha=0.7)
                plt.tight_layout()
                plt.show()
        
        return {"success": True, "trainer": trainer, "result": result}
        
    except Exception as e:
        import traceback
        print(f"❌ 训练器测试失败: {str(e)}")
        traceback.print_exc()
        return {"success": False, "error": str(e)}

# 运行训练器测试
trainer_test_results = test_trainer_factory()

## 5. 工具模块测试

测试损失函数、评估指标和模型选择工具

In [ ]:
def test_utils_modules():
    print("🔍 测试工具模块...")
    
    results = {}
    
    # 测试损失函数工具
    print("\n📊 测试损失函数工具...")
    try:
        from src.utils.loss_utils import get_loss_function
        
        # 测试获取不同类型的损失函数
        loss_functions = ['cross_entropy', 'mse', 'focal', 'rul']
        for loss_name in loss_functions:
            try:
                loss_fn = get_loss_function(loss_name)
                print(f"✅ 成功获取损失函数: {loss_name} -> {type(loss_fn).__name__}")
            except Exception as e:
                print(f"❌ 加载损失函数失败: {loss_name} - {str(e)}")
        
        results['loss_utils'] = True
    except Exception as e:
        print(f"❌ 损失函数工具测试失败: {str(e)}")
        results['loss_utils'] = False
    
    # 测试评估指标工具
    print("\n📊 测试评估指标工具...")
    try:
        from src.utils.metrics_utils import compute_metrics
        
        # 创建一些测试数据
        y_true = np.array([0, 1, 0, 1, 0, 1, 0, 1])
        y_pred = np.array([0, 1, 1, 1, 0, 0, 0, 1])
        
        # 计算分类指标
        metrics = compute_metrics('classification', y_true, y_pred)
        print("✅ 分类指标计算结果:")
        for k, v in metrics.items():
            if k != 'confusion_matrix':  # 排除混淆矩阵，它太大了
                print(f"   {k}: {v}")
        
        # 如果有混淆矩阵，绘制它
        if 'confusion_matrix' in metrics:
            cm = metrics['confusion_matrix']
            plt.figure(figsize=(5, 4))
            plt.imshow(cm, interpolation='nearest', cmap='Blues')
            plt.title('混淆矩阵')
            plt.colorbar()
            plt.xlabel('预测标签')
            plt.ylabel('真实标签')
            classes = [f"类别{i}" for i in range(cm.shape[0])]
            tick_marks = np.arange(len(classes))
            plt.xticks(tick_marks, classes)
            plt.yticks(tick_marks, classes)
            
            # 在格子中显示数字
            for i in range(cm.shape[0]):
                for j in range(cm.shape[1]):
                    plt.text(j, i, format(cm[i, j], 'd'),
                            ha="center", va="center",
                            color="white" if cm[i, j] > cm.max() / 2 else "black")
            
            plt.tight_layout()
            plt.show()
        
        results['metrics_utils'] = True
    except Exception as e:
        import traceback
        print(f"❌ 评估指标工具测试失败: {str(e)}")
        traceback.print_exc()
        results['metrics_utils'] = False
    
    # 测试模型选择工具
    print("\n📊 测试模型选择工具...")
    try:
        from src.utils.model_selection import ModelValidator, HyperparameterTuner
        print("✅ 模型选择工具导入成功")
        # 这里不做实际测试，因为模型选择需要完整的训练流程
        results['model_selection'] = True
    except Exception as e:
        print(f"❌ 模型选择工具测试失败: {str(e)}")
        results['model_selection'] = False
    
    return results

# 运行工具模块测试
utils_test_results = test_utils_modules()

## 6. 完整流程集成测试

测试从配置文件加载到完整训练流程的所有环节

In [ ]:
def test_full_pipeline():
    print("🔍 测试完整流程...")
    
    try:
        from src.Pipeline_01_default import main as pipeline_main
        
        # 设置测试配置
        config_path = default_config_path
        iterations = 1
        use_wandb = False
        notes = "测试用例运行"
        seed = 42
        
        print(f"🚀 开始执行完整流程，配置文件: {config_path}")
        
        # 执行完整流程
        results = pipeline_main(
            config_path=config_path,
            iterations=iterations,
            use_wandb=use_wandb,
            notes=notes,
            seed=seed
        )
        
        print("\n✅ 完整流程测试成功!")
        print("📊 测试结果:")
        pprint(results)
        
        return {"success": True, "results": results}
    
    except Exception as e:
        import traceback
        print(f"❌ 完整流程测试失败: {str(e)}")
        traceback.print_exc()
        return {"success": False, "error": str(e)}

# 运行完整流程测试
pipeline_test_results = test_full_pipeline()

## 7. 测试结果汇总

In [ ]:
def print_test_summary():
    test_modules = [
        ("数据工厂", 'data_test_results'),
        ("模型工厂", 'model_test_results'),
        ("任务工厂", 'task_test_results'),
        ("训练器工厂", 'trainer_test_results'),
        ("工具模块", 'utils_test_results'),
        ("完整流程", 'pipeline_test_results')
    ]
    
    print("\n" + "="*60)
    print("📋 Vbench 框架测试汇总")
    print("="*60)
    
    all_passed = True
    for name, var_name in test_modules:
        if var_name in globals():
            result = globals()[var_name]
            if isinstance(result, dict) and result.get("success", False):
                status = "✅ 通过"
            else:
                status = "❌ 失败"
                all_passed = False
        else:
            status = "❓ 未测试"
            all_passed = False
        
        print(f"{name:<15} {status}")
    
    print("="*60)
    if all_passed:
        print("🎉 所有测试通过! Vbench 框架工作正常。")
    else:
        print("⚠️ 部分测试未通过，请查看详细结果并修复问题。")
    print("="*60)

# 打印测试结果汇总
print_test_summary()

## 8. 调试和问题诊断

如果遇到问题，可以使用以下工具进行诊断

In [ ]:
def diagnose_issues():
    print("🔍 开始诊断潜在问题...")
    
    # 检查项目结构
    print("\n📂 检查项目结构...")
    essential_dirs = [
        "src", "configs", "data", "results", "save"
    ]
    for d in essential_dirs:
        path = os.path.join(project_root, d)
        if os.path.exists(path):
            print(f"✅ 目录存在: {d}")
        else:
            print(f"❌ 目录缺失: {d}")
    
    # 检查关键模块
    print("\n📦 检查关键模块...")
    essential_modules = [
        ("data_factory", "src/data_factory/__init__.py"),
        ("model_factory", "src/model_factory/__init__.py"),
        ("task_factory", "src/task_factory/__init__.py"),
        ("trainer_factory", "src/trainer_factory/__init__.py"),
        ("utils", "src/utils/__init__.py")
    ]
    for name, path in essential_modules:
        full_path = os.path.join(project_root, path)
        if os.path.exists(full_path):
            print(f"✅ 模块存在: {name}")
        else:
            print(f"❌ 模块缺失: {name} (路径: {path})")
    
    # 检查PyTorch版本和可用性
    print("\n🧩 检查PyTorch...")
    print(f"PyTorch版本: {torch.__version__}")
    print(f"CUDA可用: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA版本: {torch.version.cuda}")
        print(f"GPU设备: {torch.cuda.get_device_name(0)}")
    
    # 检查配置文件
    print("\n📄 检查配置文件...")
    if os.path.exists(default_config_path):
        with open(default_config_path, 'r') as f:
            config = yaml.safe_load(f)
        print(f"✅ 配置文件存在: {default_config_path}")
        
        # 检查配置文件中的关键部分
        required_sections = ['dataset', 'model', 'task', 'trainer']
        for section in required_sections:
            if section in config:
                if 'name' in config[section]:
                    print(f"✅ 配置部分有效: {section} -> {config[section]['name']}")
                else:
                    print(f"⚠️ 配置部分缺少name: {section}")
            else:
                print(f"❌ 配置部分缺失: {section}")
    else:
        print(f"❌ 配置文件不存在: {default_config_path}")

# 运行问题诊断
diagnose_issues()

## 9. 自定义测试

可以使用这个区域进行特定组件的测试和调试

In [ ]:
# 这里可以添加您需要的自定义测试代码
# 例如，测试特定的模型、数据集或任务

# 示例：测试特定的损失函数
try:
    from src.utils.loss_utils import FocalLoss
    
    # 创建损失函数
    focal_loss = FocalLoss(gamma=2.0)
    print(f"✅ 成功创建FocalLoss: {focal_loss}")
    
    # 创建测试输入
    inputs = torch.randn(8, 3)  # 批大小为8，3个类别
    targets = torch.tensor([0, 1, 2, 1, 0, 2, 1, 0])  # 目标类别
    
    # 计算损失
    loss = focal_loss(inputs, targets)
    print(f"计算的损失值: {loss.item()}")
    
except Exception as e:
    print(f"测试失败: {str(e)}")